In [1]:
import pydicom
import os,re
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
import SimpleITK as sitk
import math
import sys

import AssistVisualization as vis

from MyProcessDicom import *
from LocalDicomIO import *
from RegistrationMethod import *

import cv2

transform_path = "./data/9941446R/transform/"

In [2]:
file_path = "./data/OAIMTP/P9941446R/tracing_raw_ves_TH_0_P9941446R_U.swc"

In [3]:
def get_centerline(file_path):
    df = pd.read_csv(file_path, header=None, sep=' ',index_col=0, names=['vessel_id','x','y','z','undefined','last_id'])
    line = np.array([df['x'], df['y'], df['z']]).T
    return df, line

In [4]:
df1, line1 = get_centerline(file_path)
df1.head(20)

,vessel_id,x,y,z,undefined,last_id
1,0,149.0,287.0,4.11,8.00,0
2,0,149.0,287.0,8.23,8.02,1
3,0,150.0,288.0,12.34,7.82,2
4,0,150.0,288.0,16.46,7.25,3
5,0,151.0,288.0,20.57,6.70,4
6,0,153.0,290.0,24.69,5.70,5
7,0,153.0,289.0,28.80,5.75,6
8,0,153.0,288.0,32.91,5.73,7
9,0,153.0,288.0,37.03,5.81,8
10,0,153.0,288.0,41.14,5.81,9


In [5]:
file_path2 = "./data/OAIMTP/P9941446R/tracing_raw_ves_TH_5_P9941446R_U.swc"
df2, line2 = get_centerline(file_path2)
df2.head(20)

,vessel_id,x,y,z,undefined,last_id
1,0,172.0,275.0,0.00,6.18,-1
2,0,172.0,275.0,4.11,6.10,1
3,0,172.0,276.0,8.23,6.33,2
4,0,172.0,276.0,12.34,6.31,3
5,0,173.0,276.0,16.46,6.08,4
6,0,173.0,276.0,20.57,6.00,5
7,0,174.0,275.0,24.69,5.92,6
8,0,174.0,275.0,28.80,5.84,7
9,0,174.0,275.0,32.91,5.73,8
10,0,174.0,275.0,37.03,5.81,9


In [12]:
affine = sitk.ReadTransform(transform_path+"affineTP1to0.tfm")

In [7]:
def generate_regis_with_centerline(line1, line2, transform):
    points = []
    for i in range(line1.shape[0]):
        tmp_point = transform.TransformPoint((line1[i][0], line1[i][1], i))
        points.append(tmp_point)
    ans = np.array(points)
    ans[:,2] = np.round(ans[:,2])
    dx, dy = [], []
    for i in range(ans.shape[0]):
        if int(ans[i][2]) >= 75:
            break
        dx.append(ans[i][0] - line2[int(ans[i][2])][0])
        dy.append(ans[i][1] - line2[int(ans[i][2])][1])
    dx = np.array(dx)
    dy = np.array(dy)
    displacement_image = sitk.Image([384,384,dx.shape[0]], sitk.sitkVectorFloat64)
    # The only point that has any displacement is (0,0)
    for i in range(dx.shape[0]):
        displacement = (-dx[i],-dy[i],0)
        for j in range(384):
            for k in range(384):
                displacement_image[j,k,i] = displacement
    return sitk.DisplacementFieldTransform(displacement_image)

In [8]:
displacement_field_transform = generate_regis_with_centerline(line1, line2, affine)

In [9]:
# print(displacement_field_transform)
displacement_field_transform = sitk.ReadTransform(transform_path+"displacement1.tfm")

In [17]:
file_path = "./data/9941446R/TP0"
img_array, fixed_image = read_dicom_image(file_path)
file_path = "./data/9941446R/TP5"
img_array, moving_image = read_dicom_image(file_path)
show_images(fixed_image, moving_image)

75
size = (384, 384, 75)
75
size = (384, 384, 75)


interactive(children=(IntSlider(value=37, description='fixed_image_z', max=74), IntSlider(value=37, descriptio…

In [18]:
resampled = transform_img(empty_preprocess(fixed_image), empty_preprocess(moving_image), affine)

In [19]:
show_mixed_images(empty_preprocess(fixed_image), resampled)

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [20]:
resampled2 = transform_img(empty_preprocess(fixed_image), resampled, displacement_field_transform)

In [21]:
show_mixed_images(empty_preprocess(fixed_image), resampled2)

interactive(children=(IntSlider(value=37, description='image_z', max=74), FloatSlider(value=0.5, description='…

In [111]:
sitk.WriteTransform(displacement_field_transform, transform_path+"displacement1.tfm")
write_dicom_series(resampled2, "./data/9404954L/resample", "TP8to0_new")

In [17]:
sitk.WriteImage(sitk.Cast(resample_image(resampled2), sitk.sitkInt16), "./data/9404954L/resample/TH_0_P9404954L_US108.tif")

[384 384  75] [384, 384, 75]
